In [10]:
using LinearAlgebra,IntervalArithmetic,ForwardDiff

function orig_is_subset(X,Y)
    for i = 1:size(X)[1]
        if !( issubset(X[i],Y[i]) )
            return false
        end
    end
    return true
end

function orig_intersect(X,Y)
    L = []
    for i = 1:size(X)[1]
        push!(L, intersect(X[i],Y[i]))
        # typeof(intersect(X[i],Y[i])) => Interval{Float64}
    end
    return L
end

function orig_is_empty(X,Y)
    for i = 1:size(X)[1]
        if ( isempty(intersect(X[i],Y[i])) )
            return true
        end
    end
    return false
end

function orig_devide(X)
    #最も長い辺を二等分する
    max_diam=(0..0)
    max_index=0
    X1,X2=[],[]
    for (i,x) in enumerate(X)
        if diam(x)>max_diam
            max_diam=diam(x)
            max_index=i
        end
    end
    
    if diam(X[max_index])<10^(-8)
        return false
    end
    
    X1=X*1 #値をコピーする
    X2=X*1 #値をコピーする
    X1[max_index]= intersect(X[max_index],X[max_index]-radius(X[max_index]))
    X2[max_index]= intersect(X[max_index],X[max_index]+(diam(X[max_index])-radius(X[max_index])))
    return X1,X2
end


function allsol(F,X)
    cnt=0
    L = [X] #調査する区間のリスト
    S = [] #見つかった解のリスト
    
    while (size(L)[1] != 0)
        
        X = popfirst!(L) #Lから区間を一つ取り出す
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(X)) ) 
            print(X)
            println(":非存在条件を満たす")
            continue
        end
        
        c = mid.(X)
        j = ForwardDiff.jacobian(F,X)
        
        if !( orig_is_subset([(0. ..0.),(0. ..0.)], F(c)+j*(X-c)) )
            print(X)
            println(":非存在条件を満たす")
            continue
        end
    
        tmp = ForwardDiff.jacobian(F,c)
        
        R = tmp
        try
            R = inv(tmp)
        catch e
            try
                X1,X2 = orig_devide(X)
                push!(L, X1)
                push!(L, X2)
            catch e
                println(e)
            end
            print(X)
            println(":逆行列の演算に失敗")
            continue
        end
        
        M = Matrix{Float64}(I,size(R))-R*j
        K = c -R*F(c)+M*(X-c)
        @show K
        @show X
        
        if (orig_is_empty(K,X))
            print(X)
            println(":解なし")
            continue #解なし
        end
        
        if (orig_is_subset(K,X))　#解あり
            push!(S, K) #解のリストに追加
            print(X)
            println(":解あり")
            continue
        else
            print(K)
            println(X)
        end
        
        try
            print(X)
            println(":未知")
            X1,X2 = orig_devide(orig_intersect(K,X))　#K and X を2分割
            push!(L, X1)
            push!(L, X2)
        catch e
            println(e)
        end
    end
    @show L
    @show S
    return S
end

X = [(-1. ..1.),(-1. .. 1.)]

f((x1, x2)) = x1^2 + x2^2 - 1
g((x1, x2)) = x1^2 - x2^4

# f((x1,x2)) = x1 + 2*x2 +1
# g((x1,x2)) = x1^2 + 2*(x2^2) -1

# f((x1,x2)) = x1 + 2*x2 +1
# g((x1,x2)) = x1^2 + 2*(x2^2) -1

F((x,y))=[f((x,y));g((x,y))]

li=allsol(F,X)

for i in li
    @show i
end

# 境界に解がのっているとき、区間拡張？
# 重解　＝＞区間幅に制限をつける

Interval{Float64}[[-1, 1], [-1, 1]]:逆行列の演算に失敗
Interval{Float64}[[-1, 0], [-1, 1]]:逆行列の演算に失敗
Interval{Float64}[[0, 1], [-1, 1]]:逆行列の演算に失敗
K = Interval{Float64}[[-2.375, 1.29167], [-3.12501, 1.20834]]
X = Interval{Float64}[[-1, 0], [-1, 0]]
Interval{Float64}[[-2.375, 1.29167], [-3.12501, 1.20834]]Interval{Float64}[[-1, 0], [-1, 0]]
Interval{Float64}[[-1, 0], [-1, 0]]:未知
K = Interval{Float64}[[-2.375, 1.29167], [-1.20834, 3.12501]]
X = Interval{Float64}[[-1, 0], [0, 1]]
Interval{Float64}[[-2.375, 1.29167], [-1.20834, 3.12501]]Interval{Float64}[[-1, 0], [0, 1]]
Interval{Float64}[[-1, 0], [0, 1]]:未知
K = Interval{Float64}[[-1.29167, 2.375], [-3.12501, 1.20834]]
X = Interval{Float64}[[0, 1], [-1, 0]]
Interval{Float64}[[-1.29167, 2.375], [-3.12501, 1.20834]]Interval{Float64}[[0, 1], [-1, 0]]
Interval{Float64}[[0, 1], [-1, 0]]:未知
K = Interval{Float64}[[-1.29167, 2.375], [-1.20834, 3.12501]]
X = Interval{Float64}[[0, 1], [0, 1]]
Interval{Float64}[[-1.29167, 2.375], [-1.20834, 3.12501]]Interval{F

K = Interval{Float64}[[0.617508, 0.618632], [0.785811, 0.786623]]
X = Interval{Float64}[[0.594201, 0.622094], [0.771026, 0.787341]]
Interval{Float64}[[0.594201, 0.622094], [0.771026, 0.787341]]:解あり
K = Interval{Float64}[[0.617468, 0.618602], [0.785872, 0.786662]]
X = Interval{Float64}[[0.594201, 0.622094], [0.78734, 0.803654]]
Interval{Float64}[[0.594201, 0.622094], [0.78734, 0.803654]]:解なし
Interval{Float64}[[0.622093, 0.641598], [0.771026, 0.787341]]:非存在条件を満たす
Interval{Float64}[[0.622093, 0.641598], [0.78734, 0.803654]]:非存在条件を満たす
L = Vector{Interval{Float64}}[]
S = Any[Interval{Float64}[[-0.618632, -0.617508], [-0.786623, -0.785811]], Interval{Float64}[[-0.618632, -0.617508], [0.785811, 0.786623]], Interval{Float64}[[0.617508, 0.618632], [-0.786623, -0.785811]], Interval{Float64}[[0.617508, 0.618632], [0.785811, 0.786623]]]
i = Interval{Float64}[[-0.618632, -0.617508], [-0.786623, -0.785811]]
i = Interval{Float64}[[-0.618632, -0.617508], [0.785811, 0.786623]]
i = Interval{Float64}[[0.